In [ ]:
!pip install tensorflow_io
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [ ]:
import tensorflow as tf
import random
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas
import tensorflow_datasets as tfds
import time
import librosa.display as lidp
import tensorflow_io as tfio
from tensorflow import keras
from tensorflow.keras import layers
from IPython import display


In [ ]:
data, info = tfds.load('nsynth', try_gcs=True, split='train', with_info=True)
assert isinstance(data, tf.data.Dataset)
#get data

In [ ]:
n = 16
i = 0
p = data.take(n)
print(p)
fig, axes = plt.subplots(n,1,figsize=(15, 30))
for example in p:
  #print(list(example.keys()))
  wave = example["audio"]
  wave.numpy()
  #spectrogram = tf.signal.stft(wave, frame_length=1024, frame_step=256)
  #print(spectrogram)
  x = np.linspace(0,64000,64000)
  axes[i].plot(x,wave)
  spectrogram = tfio.experimental.audio.spectrogram(wave, nfft=512, window=1024, stride=256)
  plt.figure()
  plt.imshow(tf.math.log(spectrogram).numpy())
  #print(bytes.decode(example["id"].numpy()))
  #for a,b in example["instrument"].items():
    #print(a,b.numpy(),info.features["instrument"][a].names[b.numpy()])
  #print(example["pitch"].numpy(),info.features["pitch"].names[example["pitch"].numpy()])
  #for a,b in example["qualities"].items():
    #print(a,b.numpy())
  #print(example["velocity"].numpy(),info.features["velocity"].names[example["velocity"].numpy()])
  display.display(display.Audio(wave, rate=16000))
  i+=1

#print(info.features)
#check input data

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
#login your dirve so that you can save model data for later training

In [ ]:
tf.random.set_seed(22)
np.random.seed(22)

z_dim = 100
epochs = 3000000
batch_size = 256
learning_rate = 0.0002
print(len(data))
db = data.shuffle(batch_size*20).batch(batch_size).repeat()
#print(len(db))
db_iter = iter(db)
#d = db.take(1)
#for examples in d:
  #t = examples["audio"]
  #print(type(t),len(t))
  #print(examples["audio"].numpy())
#db = data
#db = db.take(batch_size)
#d = data.shuffle(batch_size).batch(batch_size).repeat()
#db_iter = iter(d)
#need of process data

In [ ]:
def make_generator_model(d,c):
  
  model = tf.keras.Sequential()
  model.add(layers.Dense(256*d, input_shape=(100,)))
  model.add(layers.ReLU())

  model.add(layers.Reshape((16,16*d)))
  assert model.output_shape == (None, 16, 16*d)

  model.add(layers.Conv1DTranspose(8*d, 25, strides=4, padding='same'))
  assert model.output_shape == (None, 64, 8*d)
  model.add(layers.ReLU())

  model.add(layers.Conv1DTranspose(4*d, 25, strides=4, padding='same'))
  assert model.output_shape == (None, 256, 4*d)
  model.add(layers.ReLU())

  model.add(layers.Conv1DTranspose(2*d, 25, strides=4, padding='same'))
  assert model.output_shape == (None, 1024, 2*d)
  model.add(layers.ReLU())

  model.add(layers.Conv1DTranspose(d, 25, strides=4, padding='same'))
  assert model.output_shape == (None, 4096, d)
  model.add(layers.ReLU())

  model.add(layers.Conv1DTranspose(c, 25, strides=4, padding='same', activation='tanh'))
  print(model.output_shape)
  assert model.output_shape == (None, 16384, c)

  return model

In [ ]:
generator = make_generator_model(64,1)

noise = tf.random.uniform(shape=[1, z_dim], minval=-1, maxval=1)
#print(noise)
generated_image = generator(noise, training=False)
#print(generated_image.shape[0])
for i in range(generated_image.shape[0]):
  x = np.linspace(0,16384,16384)
  print(generated_image[i,:,0])
  k = np.reshape(generated_image[i,:,0],(16384))
  display.display(display.Audio(k, rate=16000))
  plt.plot(x,k)
generated_image = generator(noise, training=False)
print(generated_image.shape[0])

In [ ]:
def make_discriminator_model(d,c):
  model = tf.keras.Sequential()

  model.add(layers.Conv1D(d, 25, strides=4, padding='same', input_shape=[16384,c]))
  assert model.output_shape == (None, 4096, d)
  model.add(layers.LeakyReLU(alpha=0.2))

  model.add(layers.Conv1D(2*d, 25, strides=4, padding='same'))
  assert model.output_shape == (None, 1024, 2*d)
  model.add(layers.LeakyReLU(alpha=0.2))

  model.add(layers.Conv1D(4*d, 25, strides=4, padding='same'))
  assert model.output_shape == (None, 256, 4*d)
  model.add(layers.LeakyReLU(alpha=0.2))

  model.add(layers.Conv1D(8*d, 25, strides=4, padding='same'))
  assert model.output_shape == (None, 64, 8*d)
  model.add(layers.LeakyReLU(alpha=0.2))

  model.add(layers.Conv1D(16*d, 25, strides=4, padding='same'))
  assert model.output_shape == (None, 16, 16*d)
  model.add(layers.LeakyReLU(alpha=0.2))

  model.add(layers.Flatten())
  model.add(layers.Dense(1))

  return model

In [ ]:
discriminator = make_discriminator_model(64,1)
#print(generated_image)
decision = discriminator(generated_image, training=False)
print (decision)

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(real_output, fake_output):
  real_loss = cross_entropy(tf.ones_like(real_output), real_output)
  fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
  total_loss = real_loss + fake_loss
  return total_loss

In [ ]:
def generator_loss(fake_output):
  return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(learning_rate)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate)

In [ ]:
checkpoint_dir = '/content/gdrive/My Drive/DCGAN'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
seed = tf.random.uniform(shape=[16, z_dim], minval=-1, maxval=1)
print(seed)

In [ ]:
def generate_and_save_images(model, epoch, test_input):
  predictions = model(test_input, training=False)

  fig, axes = plt.subplots(16,1,figsize=(15, 30))
  for i in range(predictions.shape[0]):
    x = np.linspace(0,16384,16384)
    #print(predictions[i,:,0])
    k = np.reshape(predictions[i,:,0],(16384))
    display.display(display.Audio(k, rate=16000))
    axes[i].plot(x,k)

  plt.savefig('/content/gdrive/My Drive/DCGAN/sound_at_epoch_{:04d}.png'.format(epoch))
  plt.show()
#check output data

In [ ]:
generator.summary()
discriminator.summary()
#check structure:

In [ ]:
print(db)

In [ ]:
#all above are needed

In [ ]:
count = 0
A = np.array([],float)
B = np.array([],float)

In [ ]:
A = np.loadtxt('/content/gdrive/My Drive/DCGAN/diserror.csv', delimiter=',')
B = np.loadtxt('/content/gdrive/My Drive/DCGAN/gerror.csv', delimiter=',')
count = len(A)

In [ ]:
print(count)

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))


In [ ]:
generator.save("/content/gdrive/My Drive/DCGAN/amodel")

In [ ]:
model = keras.models.load_model("/content/gdrive/My Drive/DCGAN/amodel")

In [ ]:
generate_and_save_images(model,-12,seed)

In [ ]:
for i in range(1):
  batch_x = next(db_iter)
  #print(batch_x)
  sound = batch_x["audio"]
  sound = sound[:,:16384]
  print(sound.shape)

In [ ]:
def saveitems(count,A,B):
  X = np.linspace(0,count,count)
  plt.grid(True)
  plt.title("Error tracking") # title
  plt.ylabel("Error") # y label
  plt.xlabel("Number of epochs") # x label
  plt.plot(X,A,color="blue",label="Discriminator loss")
  plt.plot(X,B,color="green",label="Generator loss")
  plt.legend()
  plt.savefig('/content/gdrive/My Drive/DCGAN/training.png')
  plt.show()
  np.savetxt('/content/gdrive/My Drive/DCGAN/diserror.csv', A, delimiter=',')
  np.savetxt('/content/gdrive/My Drive/DCGAN/gerror.csv', B, delimiter=',')

In [ ]:
epochs = 3001
for epoch in range(epochs):
  start = time.time()
  batch_x = next(db_iter)

  sound = batch_x["audio"]
  sound = sound[:,:16384]
  sound = tf.reshape(sound,[sound.shape[0],16384,1])
  #print(sound)
  batch_z = tf.random.uniform(shape=[batch_size, z_dim], minval=-1., maxval=1.)

  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    generated_sound = generator(batch_z, training=True)

    real_output = discriminator(sound, training=True)
    fake_output = discriminator(generated_sound, training=True)

    gen_loss = generator_loss(fake_output)
    disc_loss = discriminator_loss(real_output, fake_output)

  gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
  gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

  generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

  print(epoch, 'd loss:', float(disc_loss), 'g loss:', float(gen_loss))
  print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
  #generate_and_save_images(realgenerator,epoch,seed)
  A = np.append(A,float(disc_loss))
  B = np.append(B,float(gen_loss))
  count+=1
  if epoch%1000 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)
    saveitems(count,A,B)
  if epoch%500 == 0:
    generate_and_save_images(generator,count,seed)
#seed = tf.random.uniform(shape=[16, z_dim], minval=-1, maxval=1)
X = np.linspace(0,count,count)
plt.grid(True)
plt.title("Error tracking") # title
plt.ylabel("Error") # y label
plt.xlabel("Number of epochs") # x label
plt.plot(X,A,color="blue",label="Discriminator loss")
plt.plot(X,B,color="green",label="Generator loss")
plt.legend()
plt.savefig('/content/gdrive/My Drive/DCGAN/training.png')
plt.show()
np.savetxt('/content/gdrive/My Drive/DCGAN/diserror.csv', A, delimiter=',')
np.savetxt('/content/gdrive/My Drive/DCGAN/gerror.csv', B, delimiter=',')
#current training process(for first time)

In [ ]:
#generate_and_save_images(generator,epoch,seed)
X = np.linspace(0,count,count)
plt.grid(True)
plt.title("Error tracking") # title
plt.ylabel("Error") # y label
plt.xlabel("Number of epochs") # x label
plt.plot(X,A,color="blue",label="Discriminator loss")
plt.plot(X,B,color="green",label="Generator loss")
plt.legend()
plt.savefig('/content/gdrive/My Drive/DCGAN/training.png')
plt.show()
#check output

In [ ]:
print(A,type(A))
print(B,type(B))
print(count)
print(len(A))
#print A: Discriminator error, B: Generator error

In [ ]:
np.savetxt('/content/gdrive/My Drive/DCGAN/diserror.csv', A, delimiter=',')
np.savetxt('/content/gdrive/My Drive/DCGAN/gerror.csv', B, delimiter=',')
#save A,B to drive

In [ ]:
seed = tf.random.uniform(shape=[16, z_dim], minval=-1, maxval=1)
generate_and_save_images(generator,-1,seed)

In [ ]:
generate_and_save_images(generator,-1,seed)
#check for ouput